In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import sys
sys.path.append("/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/codes/COARE3p5/COAREalgorithm/Python/COARE3p5")
from coare35vn import *

In [2]:
def converttoDatetimeList(ds, timeVar='TIME'):
    timeArr = ds[timeVar].to_numpy()
    tval = pd.to_datetime(timeArr)
    timeSeries = np.array([datetime(dtm.year, dtm.month, dtm.day, dtm.hour, dtm.minute, dtm.second) for dtm in tval])
    ds['TIME'] = timeSeries
    return ds

In [3]:
def selectMatchingTime(ds1, ds2, ds3, ds4, timeVar1 = 'TIME', timeVar2='TIME', timeVar3 = 'TIME', timeVar4='TIME'):
    time1 = ds1[timeVar1].to_numpy()
    time2 = ds2[timeVar2].to_numpy()
    time3 = ds3[timeVar3].to_numpy()
    time4 = ds4[timeVar4].to_numpy()

    i = 0 
    j = 0
    k = 0
    l = 0 
    loop = True
    indices = []
    while loop:
        if time1[i] == time2[j] and time1[i] == time3[k] and time1[i] == time4[l]:
            indices.append([i,j,k,l])
            i+=1
            j+=1
            k+=1
            l+=1
        else:
            lowest = np.argmin([time1[i], time2[j], time3[k], time4[l]])
            #print(time1[i], time2[j], time3[k], time4[l])
            if lowest == 0:
                i += 1
            elif lowest == 1:
                j += 1
            elif lowest == 2:
                k += 1
            elif lowest == 3:
                l += 1
        
        if i == len(time1) or j==len(time2) or k == len(time3) or l == len(time4):
            loop = False
            
    indices = np.array(indices, dtype=int)
    sel_ds1 = ds1.isel(TIME = indices[:,0])
    sel_ds2 = ds2.isel(TIME = indices[:,1])
    sel_ds3 = ds3.isel(TIME = indices[:,2])
    sel_ds4 = ds4.isel(TIME = indices[:,3])

    return sel_ds1, sel_ds2, sel_ds3, sel_ds4

In [6]:
lat = 0
lon = -140

if lat < 0:
    latUnits = 'S'
else:
    latUnits = 'N'

if lon < 0:
    lonUnits = 'W'
else:
    lonUnits = 'E'

lat=abs(lat)
lon=abs(lon)

bWinds = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_WINDS_2000.nc'
bAirT = f'../../../downloads/Buoy/extractedGZ/AIRT/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_AIRT_2000.nc'
bSST = f'../../../downloads/Buoy/extractedGZ/SST/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_SST_2000.nc'
bRH = f'../../../downloads/Buoy/extractedGZ/RH/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_RH_2000.nc'

In [7]:
ds_WIND = xr.open_dataset(bWinds)
ds_SST = xr.open_dataset(bSST)
ds_AIRT = xr.open_dataset(bAirT)
ds_RH = xr.open_dataset(bRH)

ds_WIND = ds_WIND.sortby('TIME', ascending=True)
ds_SST = ds_SST.sortby('TIME', ascending=True)
ds_AIRT = ds_AIRT.sortby('TIME', ascending=True)
ds_RH = ds_RH.sortby('TIME', ascending=True)

ds_WIND = converttoDatetimeList(ds_WIND)#, timeVar='TIME')
ds_SST = converttoDatetimeList(ds_SST)#, timeVar='TIME')
ds_AIRT = converttoDatetimeList(ds_AIRT)#, timeVar='TIME')
ds_RH = converttoDatetimeList(ds_RH)#, timeVar='TIME')

In [ ]:
st = time.time()
ds1, ds2 , ds3, ds4= selectMatchingTime(ds_WIND, ds_SST, ds_AIRT, ds_RH)
et = time.time()
print(et- st)

In [ ]:
st = time.time()
allDS = xr.merge((ds1, ds2, ds3, ds4))
et = time.time()
print(et- st)

In [ ]:
allDS

In [ ]:
speed = allDS['WSPD'].sel(HEIGHT=4.0).to_numpy()
rh = allDS['RELH'].sel(HEIGHT=3.0).to_numpy()
sst = allDS['SST'].sel(DEPTH=1.0).to_numpy()
airt = allDS['AIRT'].sel(HEIGHT=3.0).to_numpy()

In [ ]:
coareOutPutArr = coare35vn(speed, airt, rh, sst, zu=4.0, zt = 3, zq = 3)

In [ ]:
U10N = coareOutPutArr[0,:]
u10 = coareOutPutArr[1,:]

In [ ]:
allDS

In [ ]:
WSPD_10N = xr.DataArray(np.array([U10N]).T, dims = ['TIME','HEIGHT'],
                                coords = {'TIME': allDS['TIME'],
                                         'HEIGHT': [10.0]},
                                attrs = {'units': 'meters/second',
                                         'long_name': '10 m neutral winds from COARE3.5',
                                         'vars_used_to_calculate': 'SST RH AIRT WSPD'})
WSPD_10 = xr.DataArray(np.array([u10]).T, dims = ['TIME','HEIGHT'],
                                coords = {'TIME': allDS['TIME'],
                                         'HEIGHT': [10.0]},
                                attrs = {'units': 'meters/second',
                                         'long_name': '10 m winds from COARE3.5',
                                         'vars_used_to_calculate': 'SST RH AIRT WSPD'})

In [ ]:
nds = xr.Dataset()
nds['WSPD_10N'] = WSPD_10N
nds['WSPD_10'] = WSPD_10

In [ ]:
nds

In [ ]:
alllDS = xr.merge((allDS, nds))

In [ ]:
alllDS

In [8]:
time1 = ds_WIND['TIME'].to_numpy()

In [15]:
np.array(abs(time1[0]- time1[1]), dtype='timedelta64[s]')

array(600, dtype='timedelta64[s]')